<font size="5">**Equivalent width of DIBs for extinction estimation**

In [ ]:
def weighted_average(x1, e1, x2, e2):
    w1 = 1 / e1**2
    w2 = 1 / e2**2

    average = (x1 * w1 + x2 * w2) / (w1 + w2)
    error = np.sqrt(1 / (w1 + w2))

    return average, error

Estimation of extinction values, where for some sources I took the spectrum of a different day since there were foton contaminations. Error is computed by propagation of errors and weighted average considers errors.

In [ ]:
import glob
import os
import warnings
import extinction
import numpy as np
from specutils import SpectralRegion
from specutils.analysis import equivalent_width
from specutils.fitting import fit_generic_continuum
def estimate_extinction_source(file):
    wl, flux=read_fits_spectrum(file)

    #Normalization around 5780
    mask_5780=(wl>5760*u.AA) & (wl<5795*u.AA)
    spec_5780 = Spectrum1D(spectral_axis=wl[mask_5780], flux=flux[mask_5780])
    with warnings.catch_warnings():
        warnings.simplefilter('ignore')
        normalized_flux_5780=spec_5780/fit_generic_continuum(spec_5780)(spec_5780.spectral_axis)
    
    #Normalization around 6614 (este está bien, no da errores)
    mask_6614=(wl>6590*u.AA) & (wl<6640*u.AA)
    spec_6614 = Spectrum1D(spectral_axis=wl[mask_6614], flux=flux[mask_6614])
    with warnings.catch_warnings():
        warnings.simplefilter('ignore')
        normalized_flux_6614=spec_6614/fit_generic_continuum(spec_6614)(spec_6614.spectral_axis)
    
    region_5780=SpectralRegion(5778*u.AA, 5783.5*u.AA)
    fname = os.path.basename(file)
    if fname.startswith("ysg_103_-1") or fname.startswith("ysg_104_-13") or fname.startswith("ysg_110_-21") or fname.startswith("ysg_95_06"):
        region_6614=SpectralRegion(6612.9*u.AA, 6615.9*u.AA) #for 103_-1, 104_-13, 110_-21, 95_06
    else:
        region_6614=SpectralRegion(6611.7*u.AA, 6615*u.AA) #for the rest

    equiv_width_5780=equivalent_width(normalized_flux_5780, regions=region_5780).value
    equiv_width_6614=equivalent_width(normalized_flux_6614, regions=region_6614).value

    equiv_width=np.mean([equiv_width_5780, equiv_width_6614])
    print(os.path.basename(file)[:-14], equiv_width)
    
    #Our stars are O, B, A type, so we consider no stellar photosphere contamination
    E_BminusV_5780=0.035+(1.978*equiv_width_5780)
    E_BminusV_6614=0.072+(3.846*equiv_width_6614)
    E_BminusV_5780err=np.sqrt(0.009**2+(equiv_width_5780*0.514)**2)
    E_BminusV_6614err=np.sqrt(0.022**2+(equiv_width_6614*1.192)**2)
    
    #Compute extinction
    Av_5780=3.1*E_BminusV_5780
    Av_6614=3.1*E_BminusV_6614
    Av_5780err=3.1*E_BminusV_5780err
    Av_6614err=3.1*E_BminusV_6614err
    print(os.path.basename(file)[:-14], "Av_5780=", Av_5780, "+-", Av_5780err, "Av_6614=", Av_6614, "+-", Av_6614err)
    mean_Av, mean_Averr=weighted_average(Av_5780, Av_5780err, Av_6614, Av_6614err)

    AG=extinction.fitzpatrick99(np.array([5822.39]), mean_Av, 3.1)
    ABP=extinction.fitzpatrick99(np.array([5035.75]), mean_Av, 3.1)
    ARP=extinction.fitzpatrick99(np.array([7619.96]), mean_Av, 3.1)

    AGerr=extinction.fitzpatrick99(np.array([5822.39]), mean_Averr, 3.1)
    ABPerr=extinction.fitzpatrick99(np.array([5822.39]), mean_Averr, 3.1)
    ARPerr=extinction.fitzpatrick99(np.array([5822.39]), mean_Averr, 3.1)
    
    return AG, AGerr, ABP, ABPerr, ARP, ARPerr